In [532]:
import numpy as np
import pandas as pd

In [533]:
df = pd.read_csv("data/inningHighlights.csv")

In [534]:
df = df[df["Inning"].isin(["T1", "B1"])]

In [535]:
df = df.groupby("Game")['Runs'].sum().reset_index()

In [536]:
df['Runs'] = df['Runs'] < 1

In [537]:
df.rename(columns={'Runs': 'NRFI'}, inplace=True)

In [538]:
games = pd.read_csv('data/games.csv')

Noticed 16 duplicate games

In [539]:
games = games.drop_duplicates().reset_index()

In [540]:
df = pd.merge(df, games, on='Game', how='inner')

In [541]:
df = df[['Game','NRFI','away','home','Date']]

In [542]:
df['year'] = pd.to_datetime(df['Date']).dt.year
df = df.drop(columns="Date")

In [543]:
df.head()

,Game,NRFI,away,home,year
0,360403107,False,NYM,KC,2016
1,360403123,True,STL,PIT,2016
2,360403130,False,TOR,TB,2016
3,360404101,True,MIN,BAL,2016
4,360404103,False,CHC,LAA,2016


In [544]:
pitchers = pd.read_csv('data/pitchersByGame.csv')

/var/folders/fd/wjhkh2294ws3fn6xqxwlk5qw0000gn/T/ipykernel_44295/2666576409.py:1: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  pitchers = pd.read_csv('data/pitchersByGame.csv')


Noticed duplicates in pitchers by game

In [545]:
pitchers = pitchers[["Pitchers", "Pitcher Id", 'Game', 'Team']]

In [546]:
team_rows = pitchers['Pitchers'] == 'TEAM'

starting_pitcher_indices = pitchers.index[team_rows] +  1

starting_pitcher_indices = starting_pitcher_indices.insert(0, 0)

starting_pitcher_indices = starting_pitcher_indices[starting_pitcher_indices < len(pitchers)]

starting_pitchers = pitchers.loc[starting_pitcher_indices].reset_index(drop=True)

In [547]:
starting_pitchers = starting_pitchers.drop_duplicates().reset_index()

Standardize names such as R.A. Dickey<br/>
Easier to take it out then deal with it and names like Michael A. Taylor at the same time<br/>
Also have to take all suffixes out now because pitchersByGame is inconsistent in using them or not

* The reason we have to merge on pitcher names and thus do all this is because the player ids come<br/>
from different data sets and do not match

In [548]:
def standardize_first_name(name):
    first, last = name.split(". ")
    return first[0] + ". " + last

suffixes_to_remove = {'Jr.', 'Sr.', 'II', 'III', 'IV', 'V'}

def remove_suffix(name):
    first, last_part = name.split(". ")
    
    last_parts = last_part.split(" ")
    
    if last_parts[-1] in suffixes_to_remove:
        last_name = " ".join(last_parts[:-1])
    else:
        last_name = last_part

    first_initial = first[0] + "."
    return f"{first_initial} {last_name}"
    

starting_pitchers['Pitchers'] = starting_pitchers["Pitchers"].apply(standardize_first_name)
starting_pitchers['Pitchers'] = starting_pitchers["Pitchers"].apply(remove_suffix)

In [549]:
pitchers_df = starting_pitchers.rename(columns={
    'Pitchers': 'pitcher_name',
    'Pitcher Id': 'pitcher_id',
    'Game': 'game_id',
    'Team': 'team'
})

df = df.rename(columns={
    'Game': 'game_id',
    'away': 'away_team',
    'home': 'home_team'
})


# Merge away pitcher
merged = df.merge(
    pitchers_df,
    left_on=['game_id', 'away_team'],
    right_on=['game_id', 'team'],
    how='left'
).rename(columns={
    'pitcher_name': 'away_pitcher',
    'pitcher_id': 'away_pitcher_id'
}).drop(columns=['team'])

# Merge home pitcher
merged = merged.merge(
    pitchers_df,
    left_on=['game_id', 'home_team'],
    right_on=['game_id', 'team'],
    how='left'
).rename(columns={
    'pitcher_name': 'home_pitcher',
    'pitcher_id': 'home_pitcher_id'
}).drop(columns=['team'])

#Reorder columns for clarity
merged = merged[[
    'game_id', 'NRFI',
    'away_team', 'away_pitcher', 'away_pitcher_id',
    'home_team', 'home_pitcher', 'home_pitcher_id','year'
]]

df = merged


In [550]:
df.head()

,game_id,NRFI,away_team,away_pitcher,away_pitcher_id,home_team,home_pitcher,home_pitcher_id,year
0,360403107,False,NYM,M. Harvey,31214,KC,E. Volquez,6401,2016
1,360403123,True,STL,A. Wainwright,5403,PIT,F. Liriano,6211,2016
2,360403130,False,TOR,M. Stroman,32815,TB,C. Archer,31003,2016
3,360404101,True,MIN,E. Santana,6280,BAL,C. Tillman,30285,2016
4,360404103,False,CHC,J. Arrieta,30145,LAA,G. Richards,30892,2016


In [551]:
version = 'v2'

In [552]:
if version == 'v1':
    pitcher_stats = pd.read_csv("data/no_filter_pitcher_stats.csv")
elif version == 'v2':
    pitcher_stats = pd.read_csv("data/reformed_data/pitching_stats_reformed.csv")

In [553]:
import unicodedata

def standardize_last_name(name):
    last_part, first = name.split(", ")
    
    last_parts = last_part.split(" ")
    
    if last_parts[-1] in suffixes_to_remove:
        last_name = " ".join(last_parts[:-1])
    else:
        last_name = last_part

    first_initial = first[0] + "."
    return f"{first_initial} {last_name}"

def remove_accents(text):
    if isinstance(text, str):
        return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text

pitcher_stats["last_name, first_name"] = pitcher_stats["last_name, first_name"].apply(standardize_last_name)

pitcher_stats.rename(columns={"last_name, first_name" : "pitcher_name"}, inplace=True)

pitcher_stats['pitcher_name'] = pitcher_stats['pitcher_name'].apply(remove_accents)

In [554]:
# many pitchers dont have these stats actually
if version == 'v2':
    pitcher_stats = pitcher_stats.drop(columns=["p_opp_batting_avg","p_opp_on_base_avg"])

In [555]:
home_stats = pitcher_stats.rename(columns=lambda col: f'home_pitcher_{col}' if col not in ['pitcher_name', 'year'] else col)
away_stats = pitcher_stats.rename(columns=lambda col: f'away_pitcher_{col}' if col not in ['pitcher_name', 'year'] else col)

df = df.merge(
    home_stats,
    left_on=['home_pitcher', 'year'],
    right_on=['pitcher_name', 'year'],
    how='left'
).drop(columns=['pitcher_name','home_pitcher_id','home_pitcher_player_id'])

df = df.merge(
    away_stats,
    left_on=['away_pitcher', 'year'],
    right_on=['pitcher_name', 'year'],
    how='left'
).drop(columns=['pitcher_name','away_pitcher_id','away_pitcher_player_id'])

More drop duplicates (stay at original number of 13379)

In [556]:
df = df.drop_duplicates(subset='game_id').reset_index()

In [557]:
df.shape

(13379, 14)

In [494]:
rows_with_nan = df[df.isnull().any(axis=1)]
rows_with_nan

,index,game_id,NRFI,away_team,away_pitcher,home_team,home_pitcher,year,home_pitcher_k_percent,home_pitcher_bb_percent,home_pitcher_p_era,away_pitcher_k_percent,away_pitcher_bb_percent,away_pitcher_p_era
7532,8089,401074871,True,NYY,D. German,BAL,H. Alberto,2019,NaN,NaN,NaN,25.8,6.6,4.03
7693,8262,401075039,True,TOR,M. Shoemaker,OAK,K. Morales,2019,NaN,NaN,NaN,22.2,8.3,1.57
7761,8336,401075108,False,TEX,T. Hearn,SEA,M. Gonzales,2019,17.0,6.5,3.99,NaN,NaN,NaN
7848,8431,401075199,True,SD,M. Strahm,ATL,C. Culberson,2019,NaN,NaN,NaN,24.2,4.5,4.71
7850,8433,401075201,False,BOS,C. Sale,CHW,J. Rondon,2019,NaN,NaN,NaN,35.6,6.0,4.40
7893,8480,401075245,False,ARI,J. Murphy,TB,B. Snell,2019,33.3,9.1,4.29,NaN,NaN,NaN
9781,10518,401077156,True,SD,M. Baez,ARI,T. Walker,2019,NaN,NaN,NaN,21.4,10.7,3.03
9790,10528,401078849,True,ARI,T. Widener,CHW,E. Santana,2019,7.8,9.4,9.45,NaN,NaN,NaN
9798,10539,401078857,False,LAA,D. Peters,SEA,T. Danish,2019,NaN,NaN,NaN,16.8,8.0,5.38
9803,10544,401078862,False,LAA,J. Beasley,SD,J. Lucchesi,2019,23.0,8.2,4.18,NaN,NaN,NaN


26 rows with names not found in our pitcher stats, lets remove these rows with nan values

In [495]:
df = df.dropna()

In [496]:
df.shape

(13353, 14)

New magic number is 13353

In [497]:
if version == 'v1': # only keep these stats in v1
    df['home_pitcher_low_sample'] = df['home_pitcher_pa'] < 100
    df['away_pitcher_low_sample'] = df['away_pitcher_pa'] < 100

Plan is to add weighted stats here at some point

In [498]:
batters_df = pd.read_csv("data/hittersByGame.csv")

/var/folders/fd/wjhkh2294ws3fn6xqxwlk5qw0000gn/T/ipykernel_44295/2603273913.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  batters_df = pd.read_csv("data/hittersByGame.csv")


In [499]:
batters_df = batters_df[["Hitters", "Game", "Team","Position","AB","BB"]]
batters_df.rename(columns={"Hitters" : "batter_name", "Game" : "game_id", }, inplace=True)

In [500]:
batters_df.head()

,batter_name,game_id,Team,Position,AB,BB
0,M. Carpenter,360403123,STL,3B,4,1
1,T. Pham,360403123,STL,LF,1,0
2,M. Adams,360403123,STL,1B,4,0
3,M. Holliday,360403123,STL,1B-LF,3,1
4,R. Grichuk,360403123,STL,CF,4,0


pitchers snuck in the top 5, drop them before fetching

In [501]:
batters_df = batters_df[batters_df['Position'] != 'P']

## Keep top 5 batters in the order
- top 5 is infiltrated by pinch hitters, must weed them out
- don't have the data to calculate total AB's so cant use that (for example McCutchen bats 3rd but was left off because he had one less AB + BB compared to players after him, likely from a sac fly)
- also tommy pham bat 2nd but was subbed out after one AB so would be left off
- no way of being totally confident on this

- lets just drop all guys with <= 1 AB + BB as that would be unlikely they were in the starting order

In [502]:
batters_df['AB'] = pd.to_numeric(batters_df['AB'], errors='coerce')
batters_df['BB'] = pd.to_numeric(batters_df['BB'], errors='coerce')

batters_df = batters_df[batters_df['AB'] + batters_df['BB'] > 1]


In [503]:
# test
batters_df['batter_order'] = batters_df.groupby(['game_id', 'Team']).cumcount() + 1
stl = batters_df[batters_df["Team"] == "STL"]
stl.head()

,batter_name,game_id,Team,Position,AB,BB,batter_order
0,M. Carpenter,360403123,STL,3B,4.0,1.0,1
2,M. Adams,360403123,STL,1B,4.0,0.0,2
3,M. Holliday,360403123,STL,1B-LF,3.0,1.0,3
4,R. Grichuk,360403123,STL,CF,4.0,0.0,4
5,S. Piscotty,360403123,STL,RF,3.0,1.0,5


In [504]:
batters_df = batters_df[batters_df["batter_name"] != "TEAM"]
batters_df['batter_name'] = batters_df["batter_name"].apply(standardize_first_name)
batters_df['batter_name'] = batters_df["batter_name"].apply(remove_suffix)

# have to remove accent too, this dataset is so inconsistent
batters_df['batter_name'] = batters_df['batter_name'].apply(remove_accents)

In [505]:
top_n = 5
top_batters = batters_df[batters_df['batter_order'] <= top_n]

batters_df = top_batters.pivot(
    index=['game_id', 'Team'],
    columns='batter_order',
    values='batter_name'
).reset_index()

In [506]:
batters_df.columns = ['game_id', 'Team'] + [f'batter_{i}' for i in range(1, top_n + 1)]

In [507]:
pd.set_option('display.max_columns', None)
# Merge home batters
df = df.merge(
    batters_df,
    left_on=['game_id', 'home_team'],
    right_on=['game_id', 'Team'],
    how='left',
    suffixes=('', '_home')
).drop(columns=['Team'])

df.rename(columns={f'batter_{i}': f'home_batter_{i}' for i in range(1, top_n + 1)}, inplace=True)

df = df.merge(
    batters_df,
    left_on=['game_id', 'away_team'],
    right_on=['game_id', 'Team'],
    how='left',
    suffixes=('', '_away')
).drop(columns=['Team'])

# Rename batter columns for away team
df.rename(columns={f'batter_{i}': f'away_batter_{i}' for i in range(1, top_n + 1)}, inplace=True)

going to keep default baseball savant stats for now, can experiment with more stats later <br/>
easy ones like OPS, OBP should be included later for sure

In [508]:
# set version of data
version = 'v2'

In [509]:
if version == 'v1':
    batter_stats = pd.read_csv("data/batter_stats.csv")
elif version == 'v2':
    batter_stats = pd.read_csv("data/reformed_data/batting_stats_reformed.csv")

savant puts both pitcher and batter data in this csv for some reason, when we merge only batters will remain

In [510]:
batter_stats["last_name, first_name"] = batter_stats["last_name, first_name"].apply(standardize_last_name)

batter_stats.rename(columns={"last_name, first_name" : "batter_name"}, inplace=True)

batter_stats['batter_name'] = batter_stats['batter_name'].apply(remove_accents)


rows_with_nan = batter_stats[batter_stats.isnull().any(axis=1)]
rows_with_nan

,batter_name,player_id,year,home_run,k_percent,bb_percent,batting_avg,slg_percent,on_base_percent,on_base_plus_slg,b_rbi,b_total_bases,woba


In [511]:
batter_stats = batter_stats.dropna()

In [512]:
for team in ['home', 'away']:
    for i in range(1, 6):
        batter_col = f'{team}_batter_{i}'
        stats = batter_stats.rename(
            columns=lambda col: f'{batter_col}_{col}' if col not in ['batter_name', 'year'] else col
        )

        df = df.merge(
            stats,
            left_on=[batter_col, 'year'],
            right_on=['batter_name', 'year'],
            how='left'
        ).drop(columns=['batter_name'])

df = df.drop_duplicates(subset='game_id').reset_index(drop=True)

In [513]:
if version == 'v1':
    batter_stats_to_agg = ['xwoba', 'woba', 'k_percent', 'bb_percent', 'barrel_batted_rate','sweet_spot_percent','hard_hit_percent','avg_best_speed','avg_hyper_speed','whiff_percent','swing_percent']
elif version == 'v2':
    batter_stats_to_agg = ['home_run', 'k_percent', 'bb_percent', 'batting_avg', 'slg_percent','on_base_percent','on_base_plus_slg','b_rbi','b_total_bases','woba']

for stat in batter_stats_to_agg:
    df[f'home_top5_avg_{stat}'] = df[[f'home_batter_{i}_{stat}' for i in range(1, 6)]].mean(axis=1)
    df[f'away_top5_avg_{stat}'] = df[[f'away_batter_{i}_{stat}' for i in range(1, 6)]].mean(axis=1)

### Let's think about training our data, we can add stats to optimize later

- recent performance
- specific performance (how hitters perform against specific pitchers)
- weather + stadium
- weighted stats based on sample size

### But first lets get overall team nrfi percentage as that must be a big indicator

Going to scrape teamrankings.com for the data


In [514]:
import pandas as pd
import time, random, os

# Load data if already scraped
if os.path.exists('data/first-inning-runs-per-game.csv'):
    first_inning_rpg_df = pd.read_csv('data/first-inning-runs-per-game.csv', index_col=0)
else:
    # List of seasons to scrape
    seasons = ['2016', '2017', '2018', '2019', '2020', '2021']
    first_inning_rpg_df = pd.DataFrame()

    for season in seasons:
        url = f'https://www.teamrankings.com/mlb/stat/1st-inning-runs-per-game?date={season}-11-10'
        
        try:
            tables = pd.read_html(url, header=0)
            if tables:
                season_df = tables[0]
                season_df['Season'] = season  # Add season column
                first_inning_rpg_df = pd.concat([first_inning_rpg_df, season_df], ignore_index=True)
                print(f"Scraped season {season}")
            else:
                print(f"No table found for season {season}")
        except Exception as e:
            print(f"Failed to scrape season {season}: {e}")
        
        time.sleep(random.randint(4, 6))  # Sleep to avoid rate limiting

    # Save the data
    first_inning_rpg_df.to_csv('data/first-inning-runs-per-game.csv')

In [515]:
first_inning_rpg_df.head()

,Rank,Team,2016,Last 3,Last 1,Home,Away,2015,Season,2017,2018,2019,2020,2021
0,1,Washington,0.72,0.33,0.0,0.62,0.83,0.55,2016,NaN,NaN,NaN,NaN,NaN
1,2,Boston,0.71,0.33,0.0,0.89,0.53,0.48,2016,NaN,NaN,NaN,NaN,NaN
2,3,Cincinnati,0.68,1.33,3.0,0.73,0.63,0.59,2016,NaN,NaN,NaN,NaN,NaN
3,4,Chi Cubs,0.67,1.33,1.0,0.69,0.66,0.53,2016,NaN,NaN,NaN,NaN,NaN
4,5,Arizona,0.65,2.00,0.0,0.72,0.59,0.52,2016,NaN,NaN,NaN,NaN,NaN


In [516]:
rows = []

years = [2016, 2017, 2018, 2019, 2020, 2021]
column_indices = {2016: 2, 2017: 9, 2018: 10, 2019: 11, 2020: 12, 2021: 13}

for year in years:
    col_idx = column_indices[year]

    values = first_inning_rpg_df[first_inning_rpg_df[str(year)].notna()].iloc[:30, [1, col_idx]]
    
    for _, row in values.iterrows():
        rows.append({
            "year": year,
            "team": row.iloc[0],    
            "rpg": row.iloc[1]
        })

first_inning_rpg_df = pd.DataFrame(rows)


Going to just use overall for now, can optimize later with home / away splits

Must See what abbreviations and team names we are dealing with

- so we know how to translate
- deal with relocations (oakland to sacramento)

In [517]:
print(first_inning_rpg_df["team"].drop_duplicates().tolist())

['Washington', 'Boston', 'Cincinnati', 'Chi Cubs', 'Arizona', 'Pittsburgh', 'Toronto', 'LA Dodgers', 'Colorado', 'LA Angels', 'Seattle', 'Houston', 'Miami', 'Texas', 'Milwaukee', 'Tampa Bay', 'San Diego', 'Cleveland', 'Baltimore', 'Kansas City', 'Chi Sox', 'Detroit', 'Minnesota', 'NY Mets', 'NY Yankees', 'St. Louis', 'Philadelphia', 'Atlanta', 'SF Giants', 'Sacramento']


In [518]:
print(df["away_team"].drop_duplicates().tolist())

df[df["away_team"] == 'AL']


['NYM', 'STL', 'TOR', 'MIN', 'CHC', 'SF', 'CHW', 'SEA', 'WSH', 'PHI', 'LAD', 'COL', 'BOS', 'HOU', 'DET', 'TEX', 'MIA', 'TB', 'CLE', 'NYY', 'OAK', 'PIT', 'SD', 'BAL', 'LAA', 'CIN', 'KC', 'ATL', 'MIL', 'ARI', 'NL', 'AL']


,index,game_id,NRFI,away_team,away_pitcher,home_team,home_pitcher,year,home_pitcher_k_percent,home_pitcher_bb_percent,home_pitcher_p_era,away_pitcher_k_percent,away_pitcher_bb_percent,away_pitcher_p_era,home_batter_1,home_batter_2,home_batter_3,home_batter_4,home_batter_5,away_batter_1,away_batter_2,away_batter_3,away_batter_4,away_batter_5,home_batter_1_player_id,home_batter_1_home_run,home_batter_1_k_percent,home_batter_1_bb_percent,home_batter_1_batting_avg,home_batter_1_slg_percent,home_batter_1_on_base_percent,home_batter_1_on_base_plus_slg,home_batter_1_b_rbi,home_batter_1_b_total_bases,home_batter_1_woba,home_batter_2_player_id,home_batter_2_home_run,home_batter_2_k_percent,home_batter_2_bb_percent,home_batter_2_batting_avg,home_batter_2_slg_percent,home_batter_2_on_base_percent,home_batter_2_on_base_plus_slg,home_batter_2_b_rbi,home_batter_2_b_total_bases,home_batter_2_woba,home_batter_3_player_id,home_batter_3_home_run,home_batter_3_k_percent,home_batter_3_bb_percent,home_batter_3_batting_avg,home_batter_3_slg_percent,home_batter_3_on_base_percent,home_batter_3_on_base_plus_slg,home_batter_3_b_rbi,home_batter_3_b_total_bases,home_batter_3_woba,home_batter_4_player_id,home_batter_4_home_run,home_batter_4_k_percent,home_batter_4_bb_percent,home_batter_4_batting_avg,home_batter_4_slg_percent,home_batter_4_on_base_percent,home_batter_4_on_base_plus_slg,home_batter_4_b_rbi,home_batter_4_b_total_bases,home_batter_4_woba,home_batter_5_player_id,home_batter_5_home_run,home_batter_5_k_percent,home_batter_5_bb_percent,home_batter_5_batting_avg,home_batter_5_slg_percent,home_batter_5_on_base_percent,home_batter_5_on_base_plus_slg,home_batter_5_b_rbi,home_batter_5_b_total_bases,home_batter_5_woba,away_batter_1_player_id,away_batter_1_home_run,away_batter_1_k_percent,away_batter_1_bb_percent,away_batter_1_batting_avg,away_batter_1_slg_percent,away_batter_1_on_base_percent,away_batter_1_on_base_plus_slg,away_batter_1_b_rbi,away_batter_1_b_total_bases,away_batter_1_woba,away_batter_2_player_id,away_batter_2_home_run,away_batter_2_k_percent,away_batter_2_bb_percent,away_batter_2_batting_avg,away_batter_2_slg_percent,away_batter_2_on_base_percent,away_batter_2_on_base_plus_slg,away_batter_2_b_rbi,away_batter_2_b_total_bases,away_batter_2_woba,away_batter_3_player_id,away_batter_3_home_run,away_batter_3_k_percent,away_batter_3_bb_percent,away_batter_3_batting_avg,away_batter_3_slg_percent,away_batter_3_on_base_percent,away_batter_3_on_base_plus_slg,away_batter_3_b_rbi,away_batter_3_b_total_bases,away_batter_3_woba,away_batter_4_player_id,away_batter_4_home_run,away_batter_4_k_percent,away_batter_4_bb_percent,away_batter_4_batting_avg,away_batter_4_slg_percent,away_batter_4_on_base_percent,away_batter_4_on_base_plus_slg,away_batter_4_b_rbi,away_batter_4_b_total_bases,away_batter_4_woba,away_batter_5_player_id,away_batter_5_home_run,away_batter_5_k_percent,away_batter_5_bb_percent,away_batter_5_batting_avg,away_batter_5_slg_percent,away_batter_5_on_base_percent,away_batter_5_on_base_plus_slg,away_batter_5_b_rbi,away_batter_5_b_total_bases,away_batter_5_woba,home_top5_avg_home_run,away_top5_avg_home_run,home_top5_avg_k_percent,away_top5_avg_k_percent,home_top5_avg_bb_percent,away_top5_avg_bb_percent,home_top5_avg_batting_avg,away_top5_avg_batting_avg,home_top5_avg_slg_percent,away_top5_avg_slg_percent,home_top5_avg_on_base_percent,away_top5_avg_on_base_percent,home_top5_avg_on_base_plus_slg,away_top5_avg_on_base_plus_slg,home_top5_avg_b_rbi,away_top5_avg_b_rbi,home_top5_avg_b_total_bases,away_top5_avg_b_total_bases,home_top5_avg_woba,away_top5_avg_woba
3789,4038,370711132,True,AL,C. Sale,NL,M. Scherzer,2017,34.4,6.9,2.51,36.2,5.1,2.90,C. Blackmon,E. Inciarte,G. Stanton,J. Votto,B. Harper,J. Altuve,R. Cano,J. Ramirez,M. Moustakas,A. Judge,453568.0,37.0,18.6,9.0,0.331,0.601,0.399,1.000,104.0,387.0,0.414,542255.0,11.0,13.1,6.8,0.304,0.409,0.350,0.759,57.0,271.0,0.328,519317.0,59.0,23.6,12.3,0.281,0.631,0.376,1.007,132.0,377.0,0.410,458015.0,

Realized that some all star games are in here<br/>
lets get rid of those

In [519]:
df = df[~df["away_team"].isin(['AL', 'NL'])]
print(df["away_team"].drop_duplicates().tolist())
print(len(df["away_team"].drop_duplicates().tolist()))

['NYM', 'STL', 'TOR', 'MIN', 'CHC', 'SF', 'CHW', 'SEA', 'WSH', 'PHI', 'LAD', 'COL', 'BOS', 'HOU', 'DET', 'TEX', 'MIA', 'TB', 'CLE', 'NYY', 'OAK', 'PIT', 'SD', 'BAL', 'LAA', 'CIN', 'KC', 'ATL', 'MIL', 'ARI']
30


In [520]:
location_to_abbrev = {
    'St. Louis': 'STL',
    'NY Yankees': 'NYY',
    'NY Mets': 'NYM',
    'Chi Sox': 'CHW',
    'Chi Cubs': 'CHC',
    'LA Dodgers': 'LAD',
    'LA Angels': 'LAA',
    'Boston': 'BOS',
    'Atlanta': 'ATL',
    'Cincinnati': 'CIN',
    'Cleveland': 'CLE',
    'Colorado': 'COL',
    'Detroit': 'DET',
    'Houston': 'HOU',
    'Kansas City': 'KC',
    'Miami': 'MIA',
    'Milwaukee': 'MIL',
    'Minnesota': 'MIN',
    'Sacramento': 'OAK',  # teamrankings.com updated to sacramento while our data had not
    'Philadelphia': 'PHI',
    'Pittsburgh': 'PIT',
    'San Diego': 'SD',
    'SF Giants': 'SF',
    'Seattle': 'SEA',
    'Tampa Bay': 'TB',
    'Texas': 'TEX',
    'Toronto': 'TOR',
    'Washington': 'WSH',
    'Baltimore' : 'BAL',
    'Arizona' : 'ARI'
}

# Convert the 'Location' column to abbreviations
first_inning_rpg_df['team'] = first_inning_rpg_df['team'].map(location_to_abbrev)

In [521]:
df = df.merge(
    first_inning_rpg_df,
    how='left',
    left_on=['year', 'away_team'],
    right_on=['year', 'team'],
    suffixes=('', '_away')  # Keep away stats unmodified, rename right side
).drop(columns=['team'])  # Drop duplicated 'team' column

df = df.rename(columns=lambda col: f'away_{col}' if col == 'rpg' else col)


# Merge home team stats
df = df.merge(
    first_inning_rpg_df,
    how='left',
    left_on=['year', 'home_team'],
    right_on=['year', 'team'],
    suffixes=('', '_home')
).drop(columns=['team'])

df = df.rename(columns=lambda col: f'home_{col}' if col == 'rpg' else col)

In [522]:
df[df.isnull().any(axis=1)]

,index,game_id,NRFI,away_team,away_pitcher,home_team,home_pitcher,year,home_pitcher_k_percent,home_pitcher_bb_percent,home_pitcher_p_era,away_pitcher_k_percent,away_pitcher_bb_percent,away_pitcher_p_era,home_batter_1,home_batter_2,home_batter_3,home_batter_4,home_batter_5,away_batter_1,away_batter_2,away_batter_3,away_batter_4,away_batter_5,home_batter_1_player_id,home_batter_1_home_run,home_batter_1_k_percent,home_batter_1_bb_percent,home_batter_1_batting_avg,home_batter_1_slg_percent,home_batter_1_on_base_percent,home_batter_1_on_base_plus_slg,home_batter_1_b_rbi,home_batter_1_b_total_bases,home_batter_1_woba,home_batter_2_player_id,home_batter_2_home_run,home_batter_2_k_percent,home_batter_2_bb_percent,home_batter_2_batting_avg,home_batter_2_slg_percent,home_batter_2_on_base_percent,home_batter_2_on_base_plus_slg,home_batter_2_b_rbi,home_batter_2_b_total_bases,home_batter_2_woba,home_batter_3_player_id,home_batter_3_home_run,home_batter_3_k_percent,home_batter_3_bb_percent,home_batter_3_batting_avg,home_batter_3_slg_percent,home_batter_3_on_base_percent,home_batter_3_on_base_plus_slg,home_batter_3_b_rbi,home_batter_3_b_total_bases,home_batter_3_woba,home_batter_4_player_id,home_batter_4_home_run,home_batter_4_k_percent,home_batter_4_bb_percent,home_batter_4_batting_avg,home_batter_4_slg_percent,home_batter_4_on_base_percent,home_batter_4_on_base_plus_slg,home_batter_4_b_rbi,home_batter_4_b_total_bases,home_batter_4_woba,home_batter_5_player_id,home_batter_5_home_run,home_batter_5_k_percent,home_batter_5_bb_percent,home_batter_5_batting_avg,home_batter_5_slg_percent,home_batter_5_on_base_percent,home_batter_5_on_base_plus_slg,home_batter_5_b_rbi,home_batter_5_b_total_bases,home_batter_5_woba,away_batter_1_player_id,away_batter_1_home_run,away_batter_1_k_percent,away_batter_1_bb_percent,away_batter_1_batting_avg,away_batter_1_slg_percent,away_batter_1_on_base_percent,away_batter_1_on_base_plus_slg,away_batter_1_b_rbi,away_batter_1_b_total_bases,away_batter_1_woba,away_batter_2_player_id,away_batter_2_home_run,away_batter_2_k_percent,away_batter_2_bb_percent,away_batter_2_batting_avg,away_batter_2_slg_percent,away_batter_2_on_base_percent,away_batter_2_on_base_plus_slg,away_batter_2_b_rbi,away_batter_2_b_total_bases,away_batter_2_woba,away_batter_3_player_id,away_batter_3_home_run,away_batter_3_k_percent,away_batter_3_bb_percent,away_batter_3_batting_avg,away_batter_3_slg_percent,away_batter_3_on_base_percent,away_batter_3_on_base_plus_slg,away_batter_3_b_rbi,away_batter_3_b_total_bases,away_batter_3_woba,away_batter_4_player_id,away_batter_4_home_run,away_batter_4_k_percent,away_batter_4_bb_percent,away_batter_4_batting_avg,away_batter_4_slg_percent,away_batter_4_on_base_percent,away_batter_4_on_base_plus_slg,away_batter_4_b_rbi,away_batter_4_b_total_bases,away_batter_4_woba,away_batter_5_player_id,away_batter_5_home_run,away_batter_5_k_percent,away_batter_5_bb_percent,away_batter_5_batting_avg,away_batter_5_slg_percent,away_batter_5_on_base_percent,away_batter_5_on_base_plus_slg,away_batter_5_b_rbi,away_batter_5_b_total_bases,away_batter_5_woba,home_top5_avg_home_run,away_top5_avg_home_run,home_top5_avg_k_percent,away_top5_avg_k_percent,home_top5_avg_bb_percent,away_top5_avg_bb_percent,home_top5_avg_batting_avg,away_top5_avg_batting_avg,home_top5_avg_slg_percent,away_top5_avg_slg_percent,home_top5_avg_on_base_percent,away_top5_avg_on_base_percent,home_top5_avg_on_base_plus_slg,away_top5_avg_on_base_plus_slg,home_top5_avg_b_rbi,away_top5_avg_b_rbi,home_top5_avg_b_total_bases,away_top5_avg_b_total_bases,home_top5_avg_woba,away_top5_avg_woba,away_rpg,home_rpg
835,893,360605105,False,KC,C. Young,CLE,C. Kluber,2016,26.4,6.6,3.14,23.2,10.6,6.19,C. Santana,J. Kipnis,F. Lindor,M. Napoli,L. Chisenhall,A. Escobar,W. Merrifield,E. Hosmer,S. Perez,T. Cruz,467793.0,34.0,14.4,14.4,0.259,0.498,0.366,0.864,87.0,290.0,0.370,543401.0,23.0,21.2,8.7,0.275,0.469,0.343,0.812,82.0,286.0,0.347,596019.0,15.0,12.9,8.3,0.301,0.435,0.358,0.793,78.0,263

Based on a few rows I've looked at, these leftover 93 rows of nan contain players who are not in the batter_stats csv so we can drop them comfortably

In [523]:
df = df.dropna()
df.head()

,index,game_id,NRFI,away_team,away_pitcher,home_team,home_pitcher,year,home_pitcher_k_percent,home_pitcher_bb_percent,home_pitcher_p_era,away_pitcher_k_percent,away_pitcher_bb_percent,away_pitcher_p_era,home_batter_1,home_batter_2,home_batter_3,home_batter_4,home_batter_5,away_batter_1,away_batter_2,away_batter_3,away_batter_4,away_batter_5,home_batter_1_player_id,home_batter_1_home_run,home_batter_1_k_percent,home_batter_1_bb_percent,home_batter_1_batting_avg,home_batter_1_slg_percent,home_batter_1_on_base_percent,home_batter_1_on_base_plus_slg,home_batter_1_b_rbi,home_batter_1_b_total_bases,home_batter_1_woba,home_batter_2_player_id,home_batter_2_home_run,home_batter_2_k_percent,home_batter_2_bb_percent,home_batter_2_batting_avg,home_batter_2_slg_percent,home_batter_2_on_base_percent,home_batter_2_on_base_plus_slg,home_batter_2_b_rbi,home_batter_2_b_total_bases,home_batter_2_woba,home_batter_3_player_id,home_batter_3_home_run,home_batter_3_k_percent,home_batter_3_bb_percent,home_batter_3_batting_avg,home_batter_3_slg_percent,home_batter_3_on_base_percent,home_batter_3_on_base_plus_slg,home_batter_3_b_rbi,home_batter_3_b_total_bases,home_batter_3_woba,home_batter_4_player_id,home_batter_4_home_run,home_batter_4_k_percent,home_batter_4_bb_percent,home_batter_4_batting_avg,home_batter_4_slg_percent,home_batter_4_on_base_percent,home_batter_4_on_base_plus_slg,home_batter_4_b_rbi,home_batter_4_b_total_bases,home_batter_4_woba,home_batter_5_player_id,home_batter_5_home_run,home_batter_5_k_percent,home_batter_5_bb_percent,home_batter_5_batting_avg,home_batter_5_slg_percent,home_batter_5_on_base_percent,home_batter_5_on_base_plus_slg,home_batter_5_b_rbi,home_batter_5_b_total_bases,home_batter_5_woba,away_batter_1_player_id,away_batter_1_home_run,away_batter_1_k_percent,away_batter_1_bb_percent,away_batter_1_batting_avg,away_batter_1_slg_percent,away_batter_1_on_base_percent,away_batter_1_on_base_plus_slg,away_batter_1_b_rbi,away_batter_1_b_total_bases,away_batter_1_woba,away_batter_2_player_id,away_batter_2_home_run,away_batter_2_k_percent,away_batter_2_bb_percent,away_batter_2_batting_avg,away_batter_2_slg_percent,away_batter_2_on_base_percent,away_batter_2_on_base_plus_slg,away_batter_2_b_rbi,away_batter_2_b_total_bases,away_batter_2_woba,away_batter_3_player_id,away_batter_3_home_run,away_batter_3_k_percent,away_batter_3_bb_percent,away_batter_3_batting_avg,away_batter_3_slg_percent,away_batter_3_on_base_percent,away_batter_3_on_base_plus_slg,away_batter_3_b_rbi,away_batter_3_b_total_bases,away_batter_3_woba,away_batter_4_player_id,away_batter_4_home_run,away_batter_4_k_percent,away_batter_4_bb_percent,away_batter_4_batting_avg,away_batter_4_slg_percent,away_batter_4_on_base_percent,away_batter_4_on_base_plus_slg,away_batter_4_b_rbi,away_batter_4_b_total_bases,away_batter_4_woba,away_batter_5_player_id,away_batter_5_home_run,away_batter_5_k_percent,away_batter_5_bb_percent,away_batter_5_batting_avg,away_batter_5_slg_percent,away_batter_5_on_base_percent,away_batter_5_on_base_plus_slg,away_batter_5_b_rbi,away_batter_5_b_total_bases,away_batter_5_woba,home_top5_avg_home_run,away_top5_avg_home_run,home_top5_avg_k_percent,away_top5_avg_k_percent,home_top5_avg_bb_percent,away_top5_avg_bb_percent,home_top5_avg_batting_avg,away_top5_avg_batting_avg,home_top5_avg_slg_percent,away_top5_avg_slg_percent,home_top5_avg_on_base_percent,away_top5_avg_on_base_percent,home_top5_avg_on_base_plus_slg,away_top5_avg_on_base_plus_slg,home_top5_avg_b_rbi,away_top5_avg_b_rbi,home_top5_avg_b_total_bases,away_top5_avg_b_total_bases,home_top5_avg_woba,away_top5_avg_woba,away_rpg,home_rpg
0,0,360403107,False,NYM,M. Harvey,KC,E. Volquez,2016,16.3,8.9,5.37,18.9,6.2,4.86,A. Escobar,M. Moustakas,L. Cain,E. Hosmer,K. Morales,C. Granderson,D. Wright,Y. Cespedes,L. Duda,N. Walker,444876.0,7.0,14.1,4.0,0.261,0.350,0.292,0.642,55.0,223.0,0.278,519058.0,7.0,11.5,8.0,0.240,0.500,0.301,0.801,13.0,52.0,0.339,456715.0,9.0,19.4,7.1,0.287,0.408,0.339,0.747,56.0,162.0,0.322

Only down to 13255 at the end of it all, pretty good

# Build Model

In [524]:
# prepare df for model
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

def encode(df):
    df_encoded = df

    string_cols = df_encoded.select_dtypes(include=['object']).columns
    string_cols = string_cols.drop(["away_team", "home_team"])
    df_encoded = df_encoded.drop(columns=string_cols)


    for col in ["away_team", "home_team"]:
        df_encoded[col] = le.fit_transform(df_encoded[col])
        
    return df_encoded

df_encoded = encode(df)

In [525]:
# import seaborn as sns

# sns.pairplot(df_encoded, hue='NRFI')

## Decision Tree

In [526]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

def decisionTree(df):

    X = df.iloc[:,3:]
    y = df.NRFI

    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test


    decision_sklearn = DecisionTreeClassifier(
        max_depth=4, 
        min_samples_leaf=10,  # regularization
        max_leaf_nodes=20,    # cap complexity
        random_state=42
    )
    decision_sklearn = decision_sklearn.fit(X_train, y_train)
    y_pred = decision_sklearn.predict(X_test)

    print("Train acc:", decision_sklearn.score(X_train, y_train))
    print("Test acc:", decision_sklearn.score(X_test, y_test))
    
decisionTree(df_encoded)

Train acc: 0.5490407415391249
Test acc: 0.5247674126225799


## Random Forest

In [527]:
from sklearn.ensemble import RandomForestClassifier

def randomForest(df):
    
    X = df.iloc[:,3:]
    y = df.NRFI

    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test
    
    random_forest = RandomForestClassifier(
        n_estimators=100,
        max_depth=4,
        min_samples_leaf=10,
        class_weight='balanced',  # if needed
        random_state=42
    )
    random_forest.fit(X_train, y_train)
    print("Train acc:", random_forest.score(X_train, y_train))
    print("Test acc:", random_forest.score(X_test, y_test))
    
randomForest(df_encoded)

Train acc: 0.6330028023280879
Test acc: 0.5453859693236107


In [528]:
# importances = random_forest.feature_importances_
# important_features = pd.Series(importances, index=X_train.columns).sort_values(ascending=False).head(25).index

# X_train_reduced = X_train[important_features]
# X_test_reduced = X_test[important_features]

# rf_reduced = RandomForestClassifier(random_state=42)
# rf_reduced.fit(X_train_reduced, y_train)

# print("Train acc:", rf_reduced.score(X_train_reduced, y_train))
# print("Test acc:", rf_reduced.score(X_test_reduced, y_test))

## XGBoost

In [529]:
from xgboost import XGBClassifier

def xgboost(df):

    X = df.iloc[:,3:]
    y = df.NRFI

    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

    xgb = XGBClassifier(
        n_estimators=200,
        max_depth=4,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    )
    xgb.fit(X_train, y_train)

    print("Train acc:", xgb.score(X_train, y_train))
    print("Test acc:", xgb.score(X_test, y_test))
    
xgboost(df_encoded)


/opt/anaconda3/envs/jupyter/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [14:15:29] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Train acc: 0.8484587195516275
Test acc: 0.528539099823988


In [530]:
# from sklearn.metrics import classification_report, confusion_matrix

# y_pred = rf_reduced.predict(X_test_reduced)

# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))


In [531]:
# from sklearn.metrics import roc_auc_score

# y_pred_proba = rf_reduced.predict_proba(X_test_reduced)[:, 1]
# print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))


# Cut Down Features
- use top 5 avg stats, get rid of individual
- change to what i think are more important stats
- use weighted stats based on plate appearances (for pitchers)

In [ ]:
features_to_keep = [
    # NRFI
    'NRFI',
    
    # Teams
    'home_team', 'away_team',
    
    # Pitchers
    'home_pitcher_k_percent', 'home_pitcher_bb_percent', 'home_pitcher_p_era',
    'away_pitcher_k_percent', 'away_pitcher_bb_percent', 'away_pitcher_p_era',

    # Batter top 5 avg
    'home_top5_avg_k_percent', 'home_top5_avg_bb_percent', 'home_top5_avg_batting_avg',
    'home_top5_avg_slg_percent', 'home_top5_avg_on_base_percent', 'home_top5_avg_on_base_plus_slg',
    'home_top5_avg_b_rbi', 'home_top5_avg_b_total_bases', 'home_top5_avg_woba',
    'away_top5_avg_k_percent', 'away_top5_avg_bb_percent', 'away_top5_avg_batting_avg',
    'away_top5_avg_slg_percent', 'away_top5_avg_on_base_percent', 'away_top5_avg_on_base_plus_slg',
    'away_top5_avg_b_rbi', 'away_top5_avg_b_total_bases', 'away_top5_avg_woba',

    # Team scoring context
    'home_rpg', 'away_rpg',
]

reduced_df = df[features_to_keep]

KeyError: "['NRFIhome_team', 'home_top5_avg_k_percent', 'home_top5_avg_bb_percent', 'home_top5_avg_batting_avg', 'home_top5_avg_slg_percent', 'home_top5_avg_on_base_percent', 'home_top5_avg_on_base_plus_slg', 'home_top5_avg_b_rbi', 'home_top5_avg_b_total_bases', 'home_top5_avg_woba', 'away_top5_avg_k_percent', 'away_top5_avg_bb_percent', 'away_top5_avg_batting_avg', 'away_top5_avg_slg_percent', 'away_top5_avg_on_base_percent', 'away_top5_avg_on_base_plus_slg', 'away_top5_avg_b_rbi', 'away_top5_avg_b_total_bases', 'away_top5_avg_woba', 'home_rpg', 'away_rpg'] not in index"

In [466]:
reduced_df = encode(reduced_df)

In [467]:
decisionTree(reduced_df)

AttributeError: 'DataFrame' object has no attribute 'NRFI'